<a href="https://colab.research.google.com/github/pvanand07/GPT-Research-tool/blob/master/gpt_research_tool_main_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install -U google-generativeai -q
# qdrant utilizes quantized models for fast inference on cpu
!pip install qdrant-client[fastembed] -q

In [10]:
research_questions = [
    "Identify the industry in which Canoo ev operates, along with its size, growth rate, trends, and key players.",
    "Analyze Canoo ev's main competitors, including their market share, products or services offered, pricing strategies, and marketing efforts.",
    "Identify key trends in the ev market, including changes in consumer behavior, technological advancements, and shifts in the competitive landscape.",
    "Gather information on Canoo ev's financial performance, including its revenue, profit margins, return on investment, and expense structure."
]

In [11]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
from google.colab import userdata
import requests
from IPython.display import display
from IPython.display import Markdown
import pathlib
import textwrap
import google.generativeai as genai
from googleapiclient.discovery import build
from nltk.tokenize import sent_tokenize
import ast
import json
from qdrant_client import QdrantClient
from concurrent.futures import ThreadPoolExecutor
from functools import partial
import requests
from bs4 import BeautifulSoup


my_api_key = userdata.get('google_search_api') # Get your Google Search API key from here (free) (LIMIT:100 queries/day): https://developers.google.com/custom-search/v1/overview
my_cse_id = userdata.get('cse_id') # Obtain your Custom Search Engine ID from here (free): https://programmablesearchengine.google.com/controlpanel/all
GEMINI_API_KEY = userdata.get('gemini_api') # Get your GEMINI API key from here (free) (LIMIT:100 queries/hour) https://aistudio.google.com/app/apikey

In [13]:
def google_search(search_term, api_key, cse_id, **kwargs):
    service = build("customsearch", "v1", developerKey=api_key)
    res = service.cse().list(q=search_term, cx=cse_id, **kwargs).execute()
    return res

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def ask_gemini(question, GEMINI_API_KEY):

  genai.configure(api_key=GEMINI_API_KEY)
  model = genai.GenerativeModel('gemini-pro')
  response = model.generate_content(question)
  display(to_markdown(response.text))

  return response.text

In [14]:
def generate_search_queries_prompt(question, max_iterations=6):
    """ Generates the search queries prompt for the given question.
    Args: question (str): The question to generate the search queries prompt for
    Returns: str: The search queries prompt for the given question
    """

    return 'Follow the instructions'\
           f'1. Write {max_iterations} google search queries to search online that form an objective opinion from the following: "{question}"' \
           f'2. output only the list of search queries in the format: ["query 1", "query 2", ...].'


def generate_report_prompt(question, context, report_format="apa", total_words=1000):
    """ Generates the report prompt for the given question and research summary.
    Args: question (str): The question to generate the report prompt for
            research_summary (str): The research summary to generate the report prompt for
    Returns: str: The report prompt for the given question and research summary
    """

    return f'Using the Information: """{context}"""\n\n' \
           f'answer the following' \
           f'query : "{question}" in a detailed report --' \
           f"with facts and numbers if available and a minimum of {total_words} words.\n" \
           f"You MUST write the report in {report_format} format.\n "

In [15]:
def generate_search_queries(research_questions):
  research_queries_dict ={}
  for question in research_questions:
    question_prompt = generate_search_queries_prompt(question)
    research_queries_dict[question] = ask_gemini(question_prompt,GEMINI_API_KEY)

  return research_queries_dict


def generate_research_query_list(research_queries_dict):
  # Convert Gemini output from string to list
  research_queries_dict_list = {}
  for question, research_queries in research_queries_dict.items():
    if research_queries and research_queries.strip():  # Check if the input is not null or empty
        try:
            research_queries_dict_list[question] = ast.literal_eval(research_queries)

        except SyntaxError or ValueError:
            # If ast.literal_eval() fails due to syntax error, tokenize the string into sentences using NLTK
            research_queries_dict_list[question] = sent_tokenize(research_queries)
            if len(research_queries_dict_list[question]) == 1:
              research_queries_dict_list[question] = research_queries.split('\n')
    else:
        print("Input ask_gemini(question)-->research_queries is null or empty. Passing...")

  return research_queries_dict_list

research_queries_dict = generate_search_queries(research_questions)
research_queries_dict_list = generate_research_query_list(research_queries_dict)

> ["Canoo ev industry", "Electric vehicle industry size", "Electric vehicle industry growth rate", "Electric vehicle industry trends", "Electric vehicle key players", "Canoo ev competitors"]

> ["Canoo EV competitors market share", "Canoo EV competitors products and services", "Canoo EV competitors pricing strategies", "Canoo EV competitors marketing efforts", "Comparative analysis of Canoo EV and its competitors", "Canoo EV competitive landscape"]

> ["EV market consumer behavior trends", "EV technology advancements", "EV competitive landscape shifts", "Electric vehicle industry outlook", "EV market analysis", "Future trends in the electric vehicle market"]

> ["Canoo ev revenue", "Canoo ev profit margin", "Canoo ev return on investment", "Canoo ev expense structure", "Canoo ev financial performance", "Canoo ev financial analysis"]

In [16]:
def extract_data_from_google_search_results(research_queries_dict_list):
# Using the search queries to perform google search and extract links
  research_links_google = {}

  for question, research_queries_list in research_queries_dict_list.items():
    for research_query in research_queries_list:
      research_links_google[research_query] = google_search(research_query, my_api_key, my_cse_id, num=2)

  # Writing google search results dictionary to a JSON file
  with open("research_links_google.json", 'w') as json_file:
      json.dump(research_links_google, json_file)

  print(f"Dictionary saved as JSON file: research_links_google.json")

  return research_links_google

research_links_google = extract_data_from_google_search_results(research_queries_dict_list)

Dictionary saved as JSON file: research_links_google.json


In [17]:
def extract_links(research_links_google):
  # Load the search results from the JSON file
  # File path to the JSON file
  file_path = "/content/research_links_google.json"

  # Loading JSON file as dictionary
  with open(file_path, 'r') as json_file:
      research_links_google1 = json.load(json_file)


  topics_dict_list = [research_links_google[k] for k, v in research_links_google.items()] # 24 Topics x 2 URLS

  links_list = []
  for topic_dict in topics_dict_list:
    try:
      for item in topic_dict['items']:
        links_list.append(item['link'])
    except:
      pass

  return links_list

links_list = extract_links(research_links_google)

In [18]:
class Scraper:
    """
    Scraper class to extract the content from web page links.
    """
    def __init__(self, urls, user_agent):
        """
        Initialize the Scraper class.
        Args:
            urls (list): List of URLs to scrape.
            user_agent (str): User agent string for the HTTP header to simulate a browser request.
        """
        self.urls = urls
        self.session = requests.Session()
        self.session.headers.update({
            "User-Agent": user_agent
        })

    def run(self):
        """
        Extracts the content from the provided URLs.
        Returns:
            list: A list of dictionaries with URL and its corresponding raw content.
        """
        partial_extract = partial(self.extract_data_from_link, session=self.session)
        with ThreadPoolExecutor(max_workers=20) as executor:
            contents = executor.map(partial_extract, self.urls)
        res = [content for content in contents if content['raw_content']]
        return res

    def extract_data_from_link(self, link, session):
        """
        Extracts the data from a single web page link.
        Args:
            link (str): URL to scrape.
            session (requests.Session): Session object for making requests.
        Returns:
            dict: A dictionary with URL and its raw content.
        """
        try:
            response = session.get(link, timeout=4)
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'lxml', from_encoding=response.encoding)
                raw_content = self.scrape_text_with_bs(soup)
                if len(raw_content) < 100:
                    return {'url': link, 'raw_content': None}
                return {'url': link, 'raw_content': raw_content}
            return {'url': link, 'raw_content': None}
        except Exception as e:
            return {'url': link, 'raw_content': None}

    def scrape_text_with_bs(self, soup):
        """
        Extracts and cleans text content from a BeautifulSoup object.
        Args:
            soup (BeautifulSoup): BeautifulSoup object of a web page.
        Returns:
            str: Cleaned text content from the web page.
        """
        for script_or_style in soup(["script", "style"]):
            script_or_style.extract()  # Remove script and style elements

        text = soup.get_text(separator="\n")
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        content = "\n".join(chunk for chunk in chunks if chunk)
        return content


In [19]:
def extract_and_store_data(links_list, user_agent, output_file):
    """
    Extracts data from the provided URLs and stores the data as a JSON file.

    Args:
        research_links_google_list (dict): A dictionary with keys as topics and values as lists of URLs.
        user_agent (str): User agent string for HTTP requests.
        output_file (str): Path to the output JSON file.
    """

    # Initialize the Scraper with URLs and a user agent
    scraper = Scraper(links_list, user_agent)
    # Extract data
    data = scraper.run()

    data_dict = {}

    for item in data:
      data_dict[item['url']] = item['raw_content']
    data_dict

    # Write the data to a JSON file
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(data_dict, f, ensure_ascii=False, indent=4)

# Usage
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
output_file = 'extracted_data.json'  # Specify the path to the output file

# Call the function with the provided research links, user agent, and output file path
extract_and_store_data(links_list, user_agent, output_file)


In [20]:
import json

# File path to the JSON file
file_path = "/content/extracted_data.json"

# Loading JSON file as dictionary
with open(file_path, 'r') as json_file:
    extracted_data = json.load(json_file)
len(extracted_data)

29

In [21]:
%%time
def add_to_qdrant(extracted_data):

  # Initialize the Qdrant client with in-memory storage for demonstration purposes
  client = QdrantClient(":memory:")  # Adjust the host and port if necessary
  client.set_model("sentence-transformers/all-MiniLM-L6-v2")

  # Assuming `extracted_data` is a pre-defined dictionary with your data
  documents = []
  metadatas = []
  ids = []

  # Prepare your documents and metadata
  for url, data in extracted_data.items():
    sentences = data.split('\n')

    # Here you might need to adjust how you handle IDs to ensure they are unique integers
    # Qdrant currently requires numeric IDs
    for i, sentence in enumerate(sentences):
        documents.append(sentence)
        metadatas.append({"url": url+str(i)})
        ids.append(hash(f"{url}_{i}"))  # Example way to generate a numeric ID

  # Add documents to Qdrant
  client.add(
      collection_name="your_collection_name",
      documents=documents,
      metadata=metadatas,
      ids=ids
  )
  return client

client = add_to_qdrant(extracted_data)

100%|██████████| 83.2M/83.2M [00:01<00:00, 51.2MiB/s]


CPU times: user 2min 44s, sys: 1.03 s, total: 2min 45s
Wall time: 2min 46s


In [22]:
def query_index(client,collection_name, query_text, limit=100):
  """ Query the index with the given client and query text.
  """

  search_result = client.query(
  collection_name=collection_name,
  query_text=query_text, limit= limit
  )
  context = "".join(set(["".join(result.metadata['document']) for result in search_result]))

  return context

In [23]:
%%time
def create_report(client, research_questions):
  response = []
  display(to_markdown("# Your Research Report"))
  for question in research_questions:
    context = query_index(client, "your_collection_name", question)
    prompt = generate_report_prompt(question, context[:31000*4])
    response.append(ask_gemini(prompt, GEMINI_API_KEY))
    #response.append(ask_gemini("Write an detailed article about the question: {{" + question + "}} given the context from scraped: {{" + context+ "}}",GEMINI_API_KEY))
    #to_markdown(response)
  return response

response = create_report(client, research_questions)

> # Your Research Report

> **Canoo EV: Industry Analysis**
> 
> ### Introduction
> 
> **Industry Overview**
> 
> Canoo is an electric vehicle (EV) manufacturer that operates within the rapidly growing automotive industry. The industry is undergoing a significant shift towards electrification, driven by advancements in battery technology, government incentives, and increasing consumer demand for sustainable transportation.
> 
> **Market Size and Growth Rate**
> 
> The global EV market is estimated to be worth $287.3 billion in 2023 and is projected to reach $1.32 trillion by 2030, exhibiting a compound annual growth rate (CAGR) of 22.7% during the forecast period (2023-2030) (Grand View Research, 2023).
> 
> **Market Trends**
> 
> Several key trends are shaping the EV market:
> 
> * **Increased Battery Capacity and Range:** Advancements in battery technology are leading to longer driving ranges on a single charge, reducing range anxiety among consumers.
> * **Shorter Charging Times:** Fast-charging technology allows EVs to be charged more quickly, reducing the inconvenience of charging.
> * **Government Incentives:** Governments worldwide are offering tax breaks, subsidies, and other incentives to promote EV adoption.
> * **Growing Consumer Demand:** Consumers are becoming more environmentally conscious and are increasingly choosing EVs as a sustainable alternative to gasoline-powered vehicles.
> 
> ### Key Players
> 
> The EV market is highly competitive, with numerous established and emerging players. Some of the key players in the industry include:
> 
> * **Tesla:** The global leader in EV sales, known for its high-performance vehicles and advanced technology.
> * **BYD:** A Chinese automaker that is a major player in the global EV market, particularly in the bus and truck segments.
> * **Volkswagen:** A German automaker that is investing heavily in EV development and aims to become a major player in the market.
> * **General Motors:** An American automaker that is transitioning its portfolio towards EVs and recently announced plans to sell only EVs by 2035.
> * **Ford Motor Company:** An American automaker that is also investing heavily in EV development and has released several popular EV models.
> 
> ### Competitive Landscape
> 
> Canoo EV operates in a highly competitive market, facing competition from established automakers as well as emerging startups. The company differentiates itself through its unique business model and its focus on commercial and lifestyle vehicles.
> 
> **Canoo's Unique Value Proposition**
> 
> * **Modular "Skateboard" Platform:** Canoo has developed a proprietary "skateboard" platform that integrates the battery, electric motors, and other critical components. This allows the company to offer vehicles with different body styles and configurations, enabling it to target a wide range of customer needs.
> * **Subscription-Based Business Model:** Canoo offers its vehicles on a subscription basis, providing customers with a flexible and cost-effective way to access their vehicles without the traditional costs associated with ownership, such as down payments and monthly loan payments.
> 
> **Challenges**
> 
> Like other EV startups, Canoo faces several challenges in the competitive EV market:
> 
> * **Limited Production Capacity:** Canoo is still in the early stages of production and has not yet achieved significant economies of scale.
> * **Supply Chain Disruptions:** The COVID-19 pandemic and other global events have led to supply chain disruptions, which can impact the production and delivery of Canoo's vehicles.
> * **Intense Competition:** Canoo faces intense competition from established automakers with large production capacities and extensive distribution networks.
> 
> ### Growth Strategies
> 
> To address these challenges and capitalize on the growing EV market, Canoo is pursuing several growth strategies:
> 
> * **Expanding Production Capacity:** The company plans to increase its production capacity to meet the growing demand for its vehicles.
> * **Establishing Partnerships:** Canoo is partnering with other companies, such as Walmart and Kingbee, to expand its reach and provide its vehicles to a wider range of customers.
> * **Diversifying Product Line:** The company is developing new vehicle models, including pickup trucks and lifestyle vehicles, to expand its market reach.
> 
> ### Conclusion
> 
> Canoo EV operates in a rapidly growing and highly competitive EV market. The company's unique value proposition and growth strategies position it well to capture a share of the market, but it faces challenges that it must overcome to achieve long-term success. As the EV market continues to expand, Canoo EV has the potential to become a significant player in the industry.
> 
> ### References
> 
> * Grand View Research. (2023). **Electric Vehicle Market Size, Share & Trends Analysis Report by Vehicle Type (Passenger Cars, Buses, Light Commercial Vehicles, Heavy Commercial Vehicles), by Power Source (Battery, Hybrid), by Application, by Region, and Segment Forecast, 2023 - 2030.** https://www.grandviewresearch.com/industry-analysis/electric-vehicle-market

>  2000+% 202-”100000).S, 2, 74, 8,100, 75– 20, 202 24-US- грун, 255

> ,

> **Financial Performance of Canoo Electric Vehicles**
> 
> **Revenue**
> 
> Canoo's revenue has been steadily increasing over the past few years. In 2021, the company reported revenue of $519,000. This increased to $3,321,000 in 2022, a growth of 539%. The company's revenue is expected to continue to grow in the coming years, as it ramps up production of its electric vehicles.
> 
> **Profit Margins**
> 
> Canoo's profit margins have been negative in recent years. In 2021, the company reported a net loss of $353.81 million. This improved to a net loss of $287.8 million in 2022. The company's profit margins are expected to improve in the coming years, as it scales up production and reduces costs.
> 
> **Return on Investment**
> 
> Canoo's return on investment (ROI) has been negative in recent years. In 2021, the company reported an ROI of -73%. This improved to -57% in 2022. The company's ROI is expected to improve in the coming years, as it scales up production and reduces costs.
> 
> **Expense Structure**
> 
> Canoo's expense structure is dominated by research and development (R&D) costs. In 2022, the company reported R&D costs of $138.2 million. This represented 41% of the company's total expenses. The company's R&D costs are expected to decrease in the coming years, as it completes the development of its electric vehicles.
> 
> **Overall Financial Performance**
> 
> Canoo's financial performance has been mixed in recent years. The company has reported increasing revenue, but negative profit margins and ROI. The company's financial performance is expected to improve in the coming years, as it scales up production and reduces costs.
> 
> **Financial Outlook**
> 
> Canoo expects to continue to grow its revenue in the coming years. The company expects to achieve profitability in 2024. The company's financial outlook is based on a number of assumptions, including the continued growth of the EV market, the successful launch of its electric vehicles, and the ability to reduce costs.
> 
> **Risks**
> 
> There are a number of risks that could impact Canoo's financial performance. These risks include:
> 
> * The EV market may not grow as quickly as expected.
> * Canoo may not be able to successfully launch its electric vehicles.
> * Canoo may not be able to reduce costs as quickly as expected.
> * Canoo may face competition from other EV manufacturers.
> 
> **Conclusion**
> 
> Canoo is a high-growth EV company with a strong financial outlook. The company has a number of advantages, including its proprietary EV platform, its experienced management team, and its strong partnerships. Canoo is well-positioned to capitalize on the growing EV market and achieve long-term success.

CPU times: user 583 ms, sys: 448 ms, total: 1.03 s
Wall time: 47.1 s


In [24]:
with open("report.md", 'w') as report:
  report.write("".join(response))

# FINAL REPORT IN MARKDOWN FORMAT

**Industry Overview**

Canoo EV operates within the electric vehicle (EV) industry, a rapidly growing market driven by increasing demand for sustainable transportation solutions. The industry is characterized by advancements in EV technology, rising consumer awareness, and government initiatives promoting EV adoption.

**Market Size**

As of 2023, the global EV market is estimated to be around 12.8 million units in volume. This market is expected to grow exponentially over the next decade, reaching an estimated 100 million units by 2030.

**Growth Rate**

The EV market is experiencing a significant growth rate, projected to expand at a compound annual growth rate (CAGR) of over 20% from 2023 to 2030. This growth is fueled by factors such as rising fuel prices, environmental concerns, and increasing government support.

**Trends**

* **Shift towards electric mobility:** Governments and consumers are increasingly recognizing the benefits of electric vehicles, leading to a shift away from gasoline-powered vehicles.
* **Advancements in EV technology:** Innovations in battery technology, charging infrastructure, and EV design are enhancing EV performance, range, and affordability.
* **Growing popularity of subscription models:** Subscription-based services are gaining traction in the EV market, providing consumers with flexible and affordable access to electric vehicles.
* **Rise of electric commercial vehicles:** The adoption of electric vehicles is expanding into commercial sectors, including delivery, ride-hailing, and logistics.

**Key Players**

Major players in the EV market include:

* Tesla
* BYD
* Volkswagen
* Ford
* General Motors
* Toyota
* Hyundai
* Kia
* Canoo EV

**Canoo EV's Product Strategy**

Canoo EV differentiates itself in the market by focusing on modular platform architecture and innovative designs. The company's modular platform allows for the development of a variety of vehicle types, including delivery vans, lifestyle vehicles, and multipurpose vehicles. Canoo EV's designs prioritize space utilization, versatility, and sustainability.

**Canoo EV's Marketing Strategy**

Canoo EV employs a multi-channel marketing strategy to reach its target audience. The company utilizes a combination of online and offline channels, including:

* Digital marketing: Search engine optimization (SEO), social media, and paid advertising
* Experiential marketing: Pop-up events, test drives, and ride-and-drive experiences
* Partnerships with dealerships and ride-sharing companies
* Public relations: Media outreach and industry events

**Canoo EV's Financial Performance**

As a publicly traded company, Canoo EV reports its financial performance regularly. The company's revenue has been growing steadily, and it recently reported a significant increase in vehicle deliveries. Canoo EV's financial performance is closely tied to the overall growth and demand in the EV market.

**Canoo EV's Future Outlook**

Canoo EV is well-positioned to capitalize on the growing EV market. The company's focus on innovation, strategic partnerships, and subscription-based business model provides a competitive advantage. Canoo EV has a strong pipeline of upcoming vehicle models and a clear path to profitability.**Canoo's Competitors**

Canoo faces competition from a number of established electric vehicle manufacturers, as well as startups that are also developing new electric vehicles. Some of Canoo's key competitors include:

* **Tesla** is the world's leading electric vehicle manufacturer, with a lineup of vehicles that includes the Model S, Model 3, Model X, and Model Y. Tesla is also known for its advanced Autopilot driver assistance system and its extensive network of Supercharger charging stations.
* **Rivian** is a startup that is developing a lineup of electric pickup trucks and SUVs. Rivian's vehicles are designed to be rugged and off-road capable, and they come with a variety of features that are tailored to outdoor enthusiasts.
* **Lucid** is a startup that is developing a lineup of luxury electric vehicles. Lucid's vehicles are designed to be spacious, comfortable, and technologically advanced.
* **NIO** is a Chinese electric vehicle manufacturer that is known for its high-performance vehicles. NIO's vehicles come with a variety of features that are designed to appeal to luxury buyers, such as advanced infotainment systems and premium interiors.
* **Xpeng** is a Chinese electric vehicle manufacturer that is known for its affordable vehicles. Xpeng's vehicles come with a variety of features that are designed to appeal to budget-minded consumers, such as long range and fast charging capabilities.

**Market Share**

According to a report by CleanTechnica, Tesla is the world's leading electric vehicle manufacturer, with a market share of 21%. Rivian is the second-largest electric vehicle manufacturer, with a market share of 11%. Lucid is the third-largest electric vehicle manufacturer, with a market share of 10%. NIO is the fourth-largest electric vehicle manufacturer, with a market share of 8%. Xpeng is the fifth-largest electric vehicle manufacturer, with a market share of 7%.

**Products or Services Offered**

Canoo offers a lineup of electric vehicles that includes the Lifestyle Vehicle, the MPDV, and the Pickup Truck. The Lifestyle Vehicle is a compact electric SUV that is designed for urban commuting. The MPDV is a larger electric van that is designed for commercial use. The Pickup Truck is an electric pickup truck that is designed for both work and play.

**Pricing Strategies**

Canoo's pricing strategies vary depending on the model and trim level. The Lifestyle Vehicle starts at $34,750. The MPDV starts at $33,000. The Pickup Truck starts at $39,950.

**Promotion Strategies**

Canoo uses a variety of promotion strategies to reach its target market. These strategies include:

* **Online advertising:** Canoo uses online advertising to reach potential customers who are searching for information about electric vehicles.
* **Social media:** Canoo uses social media to connect with potential customers and share information about its products and services.
* **Public relations:** Canoo uses public relations to generate positive media coverage of its products and services.
* **Events:** Canoo participates in a variety of events, such as trade shows and auto shows, to showcase its products and services to potential customers.## Key Trends in the EV Market

The electric vehicle (EV) market is experiencing significant growth, driven by a variety of factors. Here are some key trends that are shaping the industry:

**Changes in Consumer Behavior**

* Increasing environmental consciousness: Consumers are becoming more aware of the environmental impact of their transportation choices and are seeking out ways to reduce their carbon footprint.
* Growing demand for affordable and fuel-efficient vehicles: As the cost of owning and operating an EV continues to decline, more consumers are considering them as a viable option.
* Technological advancement: The development of new battery technologies and the improvement of charging infrastructure has helped to address some of the concerns that consumers have about the practicality of owning an EV.

**Technological Advancements**

* Longer driving range: New battery technologies have extended the range of the average EV, making them more convenient for daily use.
* Faster charging times: The development of new charging technologies has reduced the time it takes to charge an EV, making them more convenient for on-the-go charging.
* Advanced safety features: As the EV market matures, manufacturers are incorporating more advanced safety features into their vehicles, including lane departure warning systems, automatic emergency braking, and adaptive cruise control.

**Shifts in the Competitive Landscape**

* The EV market is becoming increasingly competitive, with new players entering the market and established automakers investing heavily in developing their own EV offerings.
* Traditional automakers are partnering with technology companies to develop new and innovative EV technologies.

## Future Outlook

The EV market is expected to continue to grow in the coming years, driven by the increasing demand for sustainable and efficient transportation options. However, the market may face some challenges, such as the high cost of EV batteries and the need for a more robust charging infrastructure. Nevertheless, the long-term outlook for the EV market is positive, as governments and consumers continue to support the transition to a more sustainable transportation system.**Canoo EV's Financial Performance**

**Revenue**

* $1.2 million in revenue reported in the first quarter of 2022
* $25 million in revenue from sales and distribution channels as of 2023

**Profit Margins**

* Not currently profitable

**Return on Investment (ROI)**

* Financial data was not available to calculate ROI

**Expense Structure**

* **Product Development:** Focused on designing and developing electric vehicles with innovative features, such as a flat "skateboard" platform.
* **Manufacturing:** outsources vehicle production to VDL Nedcar in the Netherlands.
* **Marketing and Sales:** Uses a combination of online and offline channels to reach customers.
* **General and Administrative (G&A):** Includes salaries, rent, and other operating expenses.

**Canoo EV's Financial Performance Drivers**

**Product Strategy**

* Focus on developing modular electric vehicles that can be customized to meet various needs.
* Unique "skateboard" platform enables maximum interior space and adaptability.

**Business Model**

* Subscription-based model offers flexibility and convenience to customers.
* All-inclusive subscription includes maintenance, warranty, registration, and access to insurance and vehicle charging.

**Market Positioning**

* Target market includes both consumers and businesses.
* Emphasis on affordable and sustainable electric vehicles.

**Challenges**

* Competition from established automakers
* High development and production costs
* Limited charging infrastructure
* Supply chain disruptions

**Market Outlook**

* The electric vehicle market is expected to grow significantly in the coming years, driven by factors such as:
    * Government incentives
    * Consumer demand for sustainable transportation
    * Technological advancements and cost reductions

Canoo EV is well-positioned to benefit from this market growth by leveraging its unique product offerings and innovative business model. However, the company faces strong competition and will need to execute its strategy effectively to succeed.